Görbés HF

In [ ]:
# Imports

import numpy as np
import plotly.express as px
import pandas as pd

# reading csv

df = pd.read_csv(
    "mh_autotune-main/resources/trajectories/spiral.csv")
df

Dataframe

In [2]:
fig = px.line_3d(df, x='p_x', y='p_y', z='p_z')
fig.show()

Sebesség

In [3]:
# Velocity

df["v_x"] = (df["p_x"]-df["p_x"].shift(1))/(df["t"]-df["t"].shift(1))
df["v_y"] = (df["p_y"]-df["p_y"].shift(1))/(df["t"]-df["t"].shift(1))
df["v_z"] = (df["p_z"]-df["p_z"].shift(1))/(df["t"]-df["t"].shift(1))
df["v_abs"] = np.sqrt(df["v_x"].values**2 +
                      df["v_y"].values**2+df["v_z"].values**2)
df


fig = px.scatter_3d(df.iloc[1:, :], x='p_x', y='p_y', z='p_z', color="v_abs")
fig.show()

fig = px.scatter(df.iloc[1:, :], x='v_x', y='v_x')
fig.show()

Gyorsulás

In [4]:
# Accelaration

df["a_x"] = (df["v_x"]-df["v_x"].shift(1))/(df["t"]-df["t"].shift(1))
df["a_y"] = (df["v_y"]-df["v_y"].shift(1))/(df["t"]-df["t"].shift(1))
df["a_z"] = (df["v_z"]-df["v_z"].shift(1))/(df["t"]-df["t"].shift(1))
df["a_abs"] = np.sqrt(df["a_x"].values**2 +
                      df["a_y"].values**2+df["a_z"].values**2)
df


fig = px.scatter_3d(df.iloc[1:, :], x='p_x', y='p_y', z='p_z', color="a_abs")
fig.show()

fig = px.scatter(df.iloc[1:, :], x='a_x', y='a_x')
fig.show()

Torzió

In [5]:
# Torsion

df["da_x"] = (df["a_x"]-df["a_x"].shift(1))/(df["t"]-df["t"].shift(1))
df["da_y"] = (df["a_y"]-df["a_y"].shift(1))/(df["t"]-df["t"].shift(1))
df["da_z"] = (df["a_z"]-df["a_z"].shift(1))/(df["t"]-df["t"].shift(1))

da = df["da_x"], df["da_y"], df["da_z"]
p = df["p_x"], df["p_y"], df["p_z"]
v = df["v_x"], df["v_y"], df["v_z"]
a = df["a_x"], df["a_y"], df["a_z"]

df["tv"] = (df["v_x"]*df["a_x"]*df["da_x"]+df["v_y"]*df["a_y"]
            * df["da_y"]+df["v_z"]*df["a_z"]*df["da_z"])

df["th_x"] = (df["v_y"].values*df["a_z"].values -
              df["v_z"].values*df["a_y"].values)
df["th_y"] = (df["v_z"].values*df["a_x"].values -
              df["v_x"].values*df["a_z"].values)
df["th_z"] = (df["v_x"].values*df["a_y"].values -
              df["v_y"].values*df["a_x"].values)

df["th_abs"] = np.sqrt(df["th_x"].values**2 +
                       df["th_y"].values**2+df["th_z"].values**2)

df["t"] = df["tv"]/df["th_abs"]

fig = px.scatter_3d(df.iloc[1:, :], x='p_x', y='p_y', z='p_z', color="t")
fig.show()

Kisérő triéder

In [24]:
def kisero_trieder(df=df, pont=100):

    print("\n", pont, "-es ponthoz tartozó kisérő triéder paraméterek:")

    # Érintő

    t_x = df["v_x"].values[pont]/df["v_abs"].values[pont]
    t_y = df["v_y"].values[pont]/df["v_abs"].values[pont]
    t_z = df["v_z"].values[pont]/df["v_abs"].values[pont]
    thossz = np.sqrt(t_x**2+t_y**2+t_z**2)

    t=[t_x,t_y,t_z]

    print("\nÉrintő")

    print("\tt_x[", pont, "]=", t_x)
    print("\tt_y[", pont, "]=", t_y)
    print("\tt_z[", pont, "]=", t_z)

    print("\nVektor=", t)

    # Főnormális

    n_x = df["a_x"].values[pont]/df["a_abs"].values[pont]
    n_y = df["a_y"].values[pont]/df["a_abs"].values[pont]
    n_z = df["a_z"].values[pont]/df["a_abs"].values[pont]
    nhossz = np.sqrt(n_x**2+n_y**2+n_z**2)

    n=[n_x,n_y,n_z]

    print("\nFőnormális")

    print("\tn_x[", pont, "]=", n_x)
    print("\tn_y[", pont, "]=", n_y)
    print("\tn_z[", pont, "]=", n_z)

    print("\nVektor=", n)

    #print("\nHossz=", n)

    # Binormális

    bh_x = df["v_y"].values[pont]*df["a_z"].values[pont] - \
        df["v_z"].values[pont]*df["a_y"].values[pont]
    bh_y = df["v_z"].values[pont]*df["a_x"].values[pont] - \
        df["v_x"].values[pont]*df["a_z"].values[pont]
    bh_z = df["v_x"].values[pont]*df["a_y"].values[pont] - \
        df["v_y"].values[pont]*df["a_x"].values[pont]

    bh = np.sqrt(bh_x**2+bh_y**2+bh_z**2)

    b_x = bh_x/bh
    b_y = bh_y/bh
    b_z = bh_z/bh
    bhossz = np.sqrt(b_x**2+b_y**2+b_z**2)

    b=[b_x,b_y,b_z]

    print("\nBinormális")

    print("\tb_x[", pont, "]=", b_x)
    print("\tb_y[", pont, "]=", b_y)
    print("\tb_z[", pont, "]=", b_z)

    print("\nVektor=", b)

    #print("\nHossz=", b)

    return(t, n, b)

n=input("n=")


kisero_trieder(df=df, pont=int(n))



 25 -es ponthoz tartozó kisérő triéder paraméterek:

Érintő
	t_x[ 25 ]= -0.750759441651424
	t_y[ 25 ]= -0.352441227665531
	t_z[ 25 ]= 0.5586997778886755

Vektor= [-0.750759441651424, -0.352441227665531, 0.5586997778886755]

Főnormális
	n_x[ 25 ]= -0.8670793511994419
	n_y[ 25 ]= -0.49752810859657287
	n_z[ 25 ]= -0.025282007038045035

Vektor= [-0.8670793511994419, -0.49752810859657287, -0.025282007038045035]

Binormális
	b_x[ 25 ]= 0.4917455071295034
	b_y[ 25 ]= -0.8629184639540008
	b_z[ 25 ]= 0.11643917203937494

Vektor= [0.4917455071295034, -0.8629184639540008, 0.11643917203937494]


([-0.750759441651424, -0.352441227665531, 0.5586997778886755],
 [-0.8670793511994419, -0.49752810859657287, -0.025282007038045035],
 [0.4917455071295034, -0.8629184639540008, 0.11643917203937494])